The two-hour glucose tolerance test is a serious hassle and seriously unpleasant. Most people would not voluntarily undergo it were it not a standard and important part of prenatal care. If you were worried that somebody was prediabetic, there are a number of other tests you would perform first (glycated hemoglobin aka A1C, fasting plasma glucose, random plasma glucose). So, using other tests, can we predict when somebody should be getting one of those screening tests done?

For the two-hour glucose tolerance test, which are the values in the Pima Indians Diabetes database, <140 mg/dL is normal, 141-200 mg/dL is prediabetes, and >200 mg/dL is diabetes. The same cutoffs apply for random plasma glucose, which are the values in the Framingham Heart Study database.

In [3]:
import pandas as pd
import pandas.io.sql as sqlio
import matplotlib as plt
import seaborn as sbn
import numpy as np
import psycopg2 as pg

from sklearn.linear_model import LinearRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, average_precision_score, balanced_accuracy_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from scipy.stats import uniform
import xgboost as xgb


# Pulling data from postgres
One minor piece of data engineering done in postgres in 'setup.sql': calling TRUE for diabetes on individuals in pima db who either had outcome = 1 (diabetes) or blood glucose > 139 mg/dL (prediabetes by clinical standards).

In [14]:
connection_args = {'host': 'localhost', 
                   'dbname': 'med_data', 
                   'port': 5432}
connection = pg.connect(**connection_args)

query = """SELECT * FROM pima"""

with open('tempfile.csv', 'wb') as tmpfile:
    copy_sql = "COPY ({query}) TO STDOUT WITH CSV {head}".format(
       query=query, head="HEADER"
    )
    cursor = connection.cursor()
    cursor.copy_expert(copy_sql, tmpfile)
    tmpfile.seek(0)

    pima_data = pd.read_csv('tempfile.csv')

# Imputing missing BMI values from skin thickness
I know from my exploration that there are a lot of missing values coded as 0 in this dataset. I also know that skin thickness and BMI appear to be linearly related. This leads to an obvious question - can I use a linear regression to impute missing values of one from the other?

I'm going to base the model on BMI rather than skin thickness; one is extremely common and straightforwardish to get, while body fat measures involving skin thickness on various parts of the body are notoriously imprecise and prone to measuring error. It's only two data points that have skin thickness but lack BMI, but it's practice setting up this sort of thing.

In [ ]:
pima_data = pd.read_csv('diabetes.csv')
pima_data_LR = pima_data
pima_data_LR[['BMI', 'SkinThickness']] = pima_data_LR[['BMI', 'SkinThickness']].replace(0, np.nan)
pima_data_LR = pima_data_LR.dropna()

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(np.array(pima_data_LR['SkinThickness']).reshape(-1, 1), pima_data_LR['BMI'])
lin_reg.predict(np.array([23, 23]).reshape(-1, 1))

Honestly, this is a litte worrying; the average skin thickness in the sample is 29 and the average BMI is 32. Let's graph this and see how I feel about this then. (Just using seaborn's regplot for a quick look, as I would expect its results to be very similar to the results for sklearn's LinearRegressor and the graphing process is a lot more user-friendly).

In [ ]:
sbn.regplot('SkinThickness', 'BMI', pima_data_LR)

OK, that's actually quite reasonable. Lots of noise, but that's to be expected with a skin thickness measurement.

The below is totally a case of using a bazooka to kill a mosquito, but I want to have this kind of thing in my back pocket - using a cross-validated linear regression to impute missing values.

In [ ]:
# Place to store predictions, using code from https://towardsdatascience.com/predicting-missing-values-with-python-41e581511117
y_pred = []
y_true = []
# Masking out cells with no skin thickness data or no BMI
pima_data_BMI_impute = pima_data[(pima_data['SkinThickness'] > 0) & (pima_data['BMI'] > 0)]

# Getting k-folds
kf = KFold(n_splits=5, random_state = 36)
for train_index, test_index in kf.split(pima_data_BMI_impute):
    df_test = pima_data_BMI_impute.iloc[test_index]
    df_train = pima_data_BMI_impute.iloc[train_index]
# Setting X and y
for train_index, test_index in kf.split(pima_data_BMI_impute):
    X_train = np.array(df_train['SkinThickness']).reshape(-1, 1)     
    y_train = np.array(df_train['BMI']).reshape(-1, 1)
    X_test = np.array(df_test['SkinThickness']).reshape(-1, 1)  
    y_test = np.array(df_test['BMI']).reshape(-1, 1)
# Fit linear regression
for train_index, test_index in kf.split(pima_data_BMI_impute):
    model = LinearRegression()
    model.fit(X_train, y_train)
for train_index, test_index in kf.split(pima_data_BMI_impute):
    y_pred.append(model.predict(X_test))
    y_true.append(y_test)

In [ ]:
mean_square_error = np.sqrt(np.sum(np.square(np.array(y_true) - np.array(y_pred))))

Did not figure out how to get this piped into the pandas dataframe in the time I allotted for this sidequest; leaving it there for now.

In [19]:
outcome_series = pima_data['outcome'] == 't'
pima_data['outcome'] = outcome_series

In [20]:
pima_data[['bmi', 'skinthickness', 'diabp', 'glucose', 'insulin']] = pima_data[['bmi', 'skinthickness', 'diabp', 'glucose', 'insulin']].replace(0, np.nan)

# Setting up multiple feature sets to look into which models are most predictive with which features
After discussion with Richard, including people with diagnosable prediabetes (blood glucose >= 140 mg/dL) as diabetic, since those people would also need interventions.

The basic "diagnostic" model (i.e. what might get pitched to insurance companies) can use no more than BMI, age, blood pressure, and number of pregnancies - these are data either present or collected at every well visit. For patients with a marginal diabetes risk score, diabetes pedigree function (which I assume is some sort of weighted average of close relatives with diabetes) can be obtained with a short interview ; similarly, skin thickness can be measured easily if it is found to be more informative than BMI/more informative than BMI alone.

Glucose and insulin data is being treated as a positive control in this case. Glucose is directly used to diagnose patients as diabetic or prediabetic (see cell immediately below), and type II diabetes is typically a disease of insulin-resistance, where patients develop an elevated insulin level. If these don't turn out to be highly informative features in the training sets that include them, something is deeply wrong with my assumptions and/or code.

In [21]:
pima_data['diabetes'] = (pima_data['glucose'].gt(139, fill_value = False) | pima_data['outcome'].eq(True, fill_value = False))
pima_data

,pregnancies,glucose,diabp,skinthickness,insulin,bmi,dpf,age,outcome,diabetes
0,6,148.0,72.0,35.0,NaN,33.6,0.627,50,True,True
1,1,85.0,66.0,29.0,NaN,26.6,0.351,31,False,False
2,8,183.0,64.0,NaN,NaN,23.3,0.672,32,True,True
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,False,False
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,True,True
...,...,...,...,...,...,...,...,...,...,...
763,10,101.0,76.0,48.0,180.0,32.9,0.171,63,False,False
764,2,122.0,70.0,27.0,NaN,36.8,0.340,27,False,False
765,5,121.0,72.0,23.0,112.0,26.2,0.245,30,False,False
766,1,126.0,60.0,NaN,NaN,30.1,0.349,47,True,True


In [22]:
# Making feature sets to get information on which gives the best performance
featureset1 = ['diabp', 'bmi', 'age', 'diabetes']
featureset2 = featureset1 + ['pregnancies']
featureset3 = featureset1 + ['dpf']
featureset4 = featureset1 + ['glucose', 'insulin']
featureset5 = featureset1 + ['pregnancies', 'dpf']
featureset6 = featureset4 + ['pregnancies', 'dpf']
featureset7 = featureset2 + ['skinthickness']
featureset8 = featureset5 + ['skinthickness']

In [23]:
featureset_list = [featureset1, featureset2, featureset3, featureset4, featureset5, featureset6, featureset7, featureset8]

In [24]:
pima_data1 = pima_data[featureset1].dropna()
pima_data2 = pima_data[featureset2].dropna()
pima_data3 = pima_data[featureset3].dropna()
pima_data4 = pima_data[featureset4].dropna()
pima_data5 = pima_data[featureset5].dropna()
pima_data6 = pima_data[featureset6].dropna()
pima_data7 = pima_data[featureset7].dropna()
pima_data8 = pima_data[featureset8].dropna()
pima_data1

,diabp,bmi,age,diabetes
0,72.0,33.6,50,True
1,66.0,26.6,31,False
2,64.0,23.3,32,True
3,66.0,28.1,21,False
4,40.0,43.1,33,True
...,...,...,...,...
763,76.0,32.9,63,False
764,70.0,36.8,27,False
765,72.0,26.2,30,False
766,60.0,30.1,47,True


I've got my 8 datasets with varying features, differing in size because of dropped rows. For each of these, I need to split them into train and test; I need to hold out some uncontaminated data for final results following model tuning and cross-validation. I can already tell I'll want a pipeline set up so very badly by the end of this process. Brace yourselves, mateys, thar blows the biggest school of cells you salty dogs have ever seen!

In [25]:
featureset1X = ['diabp', 'bmi', 'age']
featureset2X = featureset1X + ['pregnancies']
featureset3X = featureset1X + ['dpf']
featureset4X = featureset1X + ['glucose', 'insulin']
featureset5X = featureset1X + ['pregnancies', 'dpf']
featureset6X = featureset4X + ['pregnancies', 'dpf']
featureset7X = featureset2X + ['skinthickness']
featureset8X = featureset5X + ['skinthickness']

In [26]:
X_train_pd1, X_test_pd1, y_train_pd1, y_test_pd1 = train_test_split(pima_data1[featureset1X], 
                                                                    pima_data1['diabetes'], 
                                                                    test_size = 0.25, 
                                                                    random_state = 17)

X_train_pd2, X_test_pd2, y_train_pd2, y_test_pd2 = train_test_split(pima_data2[featureset2X], 
                                                                    pima_data2['diabetes'], 
                                                                    test_size = 0.25, 
                                                                    random_state = 8)
X_train_pd3, X_test_pd3, y_train_pd3, y_test_pd3 = train_test_split(pima_data3[featureset3X], 
                                                                    pima_data3['diabetes'], 
                                                                    test_size = 0.25, 
                                                                    random_state = 20)
X_train_pd4, X_test_pd4, y_train_pd4, y_test_pd4 = train_test_split(pima_data4[featureset4X], 
                                                                    pima_data4['diabetes'], 
                                                                    test_size = 0.25, 
                                                                    random_state = 36)
X_train_pd5, X_test_pd5, y_train_pd5, y_test_pd5 = train_test_split(pima_data5[featureset5X], 
                                                                    pima_data5['diabetes'], 
                                                                    test_size = 0.25, 
                                                                    random_state = 79)
X_train_pd6, X_test_pd6, y_train_pd6, y_test_pd6 = train_test_split(pima_data6[featureset6X], 
                                                                    pima_data6['diabetes'], 
                                                                    test_size = 0.25, 
                                                                    random_state = 11)
X_train_pd7, X_test_pd7, y_train_pd7, y_test_pd7 = train_test_split(pima_data7[featureset7X], 
                                                                    pima_data7['diabetes'], 
                                                                    test_size = 0.25, 
                                                                    random_state = 57)
X_train_pd8, X_test_pd8, y_train_pd8, y_test_pd8 = train_test_split(pima_data8[featureset8X], 
                                                                    pima_data8['diabetes'], 
                                                                    test_size = 0.25, 
                                                                    random_state = 38)

In [27]:
X_test_pd8

,diabp,bmi,age,pregnancies,dpf,skinthickness
194,55.0,24.4,42,8,0.136,20.0
121,64.0,34.2,24,6,0.260,39.0
721,66.0,38.1,21,1,0.289,36.0
742,58.0,28.5,22,1,0.219,18.0
712,62.0,41.2,38,10,0.441,36.0
...,...,...,...,...,...,...
282,88.0,32.4,37,7,0.262,15.0
368,86.0,27.5,22,3,0.306,16.0
161,74.0,37.2,45,7,0.204,40.0
150,74.0,37.4,24,1,0.399,50.0


# Logistic regression CV
Fitting logistic regressors to various feature sets with 5-fold CV. Coefficients being entered into an Excel spreadsheet so I can tab over to another program that has them always at the ready.

In [28]:
lrcv = LogisticRegressionCV(cv = 5, scoring = 'recall_weighted', class_weight = 'balanced')
lrcv.fit(X_train_pd1, y_train_pd1)
lrcv.score(X_train_pd1, y_train_pd1)

/Users/chesh1/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1920: ChangedBehaviorWarning: The long-standing behavior to use the accuracy score has changed. The scoring parameter is now used. This warning will disappear in version 0.22.
  ChangedBehaviorWarning)


0.684981684981685

In [40]:
X_train_pd1.head()

,diabp,bmi,age
600,88.0,27.1,24
493,70.0,28.9,45
213,65.0,42.6,24
358,74.0,35.3,48
530,60.0,29.8,22


In [ ]:
lrcv.coef_

In [ ]:
lrcv.fit(X_train_pd2, y_train_pd2)
lrcv.score(X_train_pd2, y_train_pd2)

In [ ]:
lrcv.coef_

In [ ]:
lrcv.fit(X_train_pd3, y_train_pd3)
lrcv.score(X_train_pd3, y_train_pd3)

In [ ]:
lrcv.coef_

In [ ]:
lrcv.fit(X_train_pd4, y_train_pd4)
lrcv.score(X_train_pd4, y_train_pd4)

In [ ]:
lrcv.coef_

In [ ]:
lrcv.fit(X_train_pd5, y_train_pd5)
lrcv.score(X_train_pd5, y_train_pd5)

In [ ]:
lrcv.coef_

In [ ]:
lrcv.fit(X_train_pd6, y_train_pd6)
lrcv.score(X_train_pd6, y_train_pd6)

In [ ]:
lrcv.coef_

In [ ]:
lrcv.fit(X_train_pd7, y_train_pd7)
lrcv.score(X_train_pd7, y_train_pd7)

In [ ]:
lrcv.coef_

In [ ]:
lrcv.fit(X_train_pd8, y_train_pd8)
lrcv.score(X_train_pd8, y_train_pd8)

In [ ]:
lrcv.coef_

In [29]:
import pickle
with open('linreg1.pkl', 'wb') as fp:
    pickle.dump(lrcv, fp)

# Further work with logistic regression
Glucose, as expected, was extremely important for logistic regression in any feature set that included it. Surprisingly, insulin wasn't similarly useful; it may be that elevated insulin levels show up later in disease/later in uncontrolled disease than elevated glucose does.

Prior to presenting work, it will be necessary to scale coefficients relative to training data. I initially thought that diabetes pedigree function was extremely important, but comparison of model scores for models with and without it indicated it might not be providing much benefit; it spans a much smaller range than other measurements, hence the much larger coefficient.

In this set of models, glucose+insulin provided the greatest score increase (0.18, avg of 2 comparisons), followed by skin thickness (0.03, avg of  2 comparisons), diabetes pedigree function (0.02, avg of 3 comparisons), and number of pregnancies (-0.01, avg of 3 comparisons). At this point, I doubt that any of the score increases are statistically significant; certainly none besides the glucose+insulin score increase are.

# k-Nearest Neighbors: are there complex decision boundaries?
As we know, logistic regression isn't good with data that are separable in a non-linear fashion. KNN, however, can be excellent at that, given scaled data which is separable within the dimensions of the feature space.

I expect that the problem is not a case of complex decision boundaries but rather one of data overlap; many people who do not currently have type II diabetes will go on to develop type II diabetes at a later point, and others who look like they are "at risk" will stave it off through diet, exercise, and other actions not captured by the features gathered. Because of this, I expect KNN to perform relatively poorly on these data. However, if KNN does perform better than logistic regression, I will need to do some more examination of the dataset.

In [30]:
# Create stratified 5-fold partitioning of training set for KNN CV
skf = StratifiedKFold(n_splits = 5)
scaler = StandardScaler()
kneighbors = KNeighborsClassifier(weights = 'distance')
acc_bal_scores = []
wt_recall_scores = []
for train, val in skf.split(X_train_pd1, y_train_pd1):
    # Fit scaler on training partitions
    scaler.fit(X_train_pd1.iloc[train])
    # Scale training and validation partitions
    scaler.transform(X_train_pd1.iloc[train])
    scaler.transform(X_train_pd1.iloc[val])
    # Fit KNN
    kneighbors.fit(X_train_pd1.iloc[train], y_train_pd1.iloc[train])
    # Get KNN score
    kpred = kneighbors.predict(X_train_pd1.iloc[val])
    acc_bal_scores.append(balanced_accuracy_score(y_train_pd1.iloc[val], kpred))
    wt_recall_scores.append(recall_score(y_train_pd1.iloc[val], kpred, average = 'weighted'))
with open('knn1.pkl', 'wb') as fp:
    pickle.dump(kneighbors, fp)

/Users/chesh1/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/chesh1/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  # This is added back by InteractiveShellApp.init_path()
/Users/chesh1/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  if sys.path[0] == '':
/Users/chesh1/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/chesh1/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DataConver

In [ ]:
wt_recall_scores

Holy hannah that's going to be an unbearable amount of manual updating. Let's pack this inside another loop, shall we?

In [ ]:
sets_to_test = [(X_train_pd2, y_train_pd2), (X_train_pd3, y_train_pd3), (X_train_pd4, y_train_pd4), (X_train_pd5, y_train_pd5), 
               (X_train_pd6, y_train_pd6), (X_train_pd7, y_train_pd7), (X_train_pd8, y_train_pd8)]
acc_bal_dict = {}
wt_recall_dict = {}
count = 2
for i in sets_to_test:
    acc_bal_scores = []
    wt_recall_scores = []
    for train, val in skf.split(i[0], i[1]):
        # Fit scaler on training partitions
        scaler.fit(i[0].iloc[train])
        # Scale training and validation partitions
        scaler.transform(i[0].iloc[train])
        scaler.transform(i[0].iloc[val])
        # Fit KNN
        kneighbors.fit(i[0].iloc[train], i[1].iloc[train])
        # Get KNN score
        kpred = kneighbors.predict(i[0].iloc[val])
        acc_bal_scores.append(balanced_accuracy_score(i[1].iloc[val], kpred))
        wt_recall_scores.append(recall_score(i[1].iloc[val], kpred, average = 'weighted'))
    acc_bal_dict[count] = acc_bal_scores
    wt_recall_dict[count] = wt_recall_scores
    count += 1

In [ ]:
acc_bal_dict

In [ ]:
wt_recall_dict

In [ ]:
# Is it that the boundaries are really complex? Lowering number of neighbors to see.
kneighbors = KNeighborsClassifier(n_neighbors = 2, weights = 'distance')
acc_bal_scores = []
wt_recall_scores = []
for train, val in skf.split(X_train_pd1, y_train_pd1):
    # Fit scaler on training partitions
    scaler.fit(X_train_pd1.iloc[train])
    # Scale training and validation partitions
    scaler.transform(X_train_pd1.iloc[train])
    scaler.transform(X_train_pd1.iloc[val])
    # Fit KNN
    kneighbors.fit(X_train_pd1.iloc[train], y_train_pd1.iloc[train])
    # Get KNN score
    kpred = kneighbors.predict(X_train_pd1.iloc[val])
    acc_bal_scores.append(balanced_accuracy_score(y_train_pd1.iloc[val], kpred))
    wt_recall_scores.append(recall_score(y_train_pd1.iloc[val], kpred, average = 'weighted'))
acc_bal_scores

# KNN results:
As expected, KNN is doing worse than logistic regression (by 5-10% in the tests above). Lowering k led to worse performance, not better. This supports my intuition that the problem is the gradual shading from not diabetic to prediabetic to diabetic, not some kind of complex boundary shapes.

Conclusion: KNN is most certainly not the right model type for this problem. Slightly duh.

# Sidequest: VIF!
Variance inflation factor is something that I keep running into in my readings and various attempts to figure out what's going on with my code. Out of curiousity and with a certain sense of dread, let's check it out!

In [ ]:
# Without skin thickness to preserve maximum amount of data
vif_test = pima_data[['Pregnancies', 'Glucose', 'Insulin', 'BMI', 'BloodPressure', 'DiabetesPedigreeFunction', 'Age', 'Outcome']].dropna()
vif_test = add_constant(vif_test)
pd.Series([variance_inflation_factor(vif_test.values, i) 
               for i in range(vif_test.shape[1])], 
              index=vif_test.columns)

In [ ]:
# With skin thickness; assumption is that skin thickness and BMI are collinear by this measure
vif_test = pima_data[['Pregnancies', 'Glucose', 'Insulin', 'SkinThickness', 'BMI', 'BloodPressure', 'DiabetesPedigreeFunction', 'Age', 'Outcome']].dropna()
vif_test = add_constant(vif_test)
pd.Series([variance_inflation_factor(vif_test.values, i) 
               for i in range(vif_test.shape[1])], 
              index=vif_test.columns)

Code shamelessly adapted from: https://stackoverflow.com/questions/42658379/variance-inflation-factor-in-python
Collinearity isn't a concern, yay! Also, now I know how to get this information from a dataframe in a few lines of code!

# Random Forest
May come back to SVM just in order to play with them; logreg indicates that classes aren't separable in any kind of linear way, so SVMs are unlikely to give performance that's much higher than logreg.

What do we do when relationships are complicated? Random forest! (Or XGBoost, but that's coming up next.)

In [ ]:
forest = RandomForestClassifier(class_weight = 'balanced')
sets_to_test = [(X_train_pd1, y_train_pd1), (X_train_pd2, y_train_pd2), (X_train_pd3, y_train_pd3), 
                (X_train_pd4, y_train_pd4), (X_train_pd5, y_train_pd5), (X_train_pd6, y_train_pd6), 
                (X_train_pd7, y_train_pd7), (X_train_pd8, y_train_pd8)]
count = 1
acc_bal_dict = {}
wt_recall_dict = {}
for i in sets_to_test:
    acc_bal_scores = []
    wt_recall_scores = []
    for train, val in skf.split(i[0], i[1]):
        # Fit forest
        forest.fit(i[0].iloc[train], i[1].iloc[train])
        # Get score
        y_pred = forest.predict(i[0].iloc[val])
        acc_bal_scores.append(balanced_accuracy_score(i[1].iloc[val], y_pred))
        wt_recall_scores.append(recall_score(i[1].iloc[val], y_pred, average = 'weighted'))
    acc_bal_dict[count] = acc_bal_scores
    wt_recall_dict[count] = wt_recall_scores
    count += 1

In [32]:
forest = RandomForestClassifier(class_weight = 'balanced')
for train, val in skf.split(X_train_pd1, y_train_pd1):
    # Fit forest
    forest.fit(X_train_pd1.iloc[train], y_train_pd1.iloc[train])
    # Get  score
    y_pred = forest.predict(X_train_pd1.iloc[val])
with open('forest1.pkl', 'wb') as fp:
    pickle.dump(forest, fp)

/Users/chesh1/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [ ]:
wt_recall_dict

Random forest isn't performing as well as I would have hoped, given what I know about it. Of course, I just jumped right in without considering how to deal with the class imbalance, so shame on me.

Changing class_weight on forest to = 'balanced' had little effect on model performance; some sets were better and others were worse, though the difference was certainly not statistically significant.

# Boosting into orbit! AdaBoost via EasyEnsebleClassifier
This is a very small dataset, so the relative slowness of AdaBoost isn't going to be a major consideration. Also, there are fewer parameters to tune for AdaBoost, so this will be somewhat less challenging to tune than XGBoost. Feature importance can also be output, which may help me understand what's going on under the hood. Noise is known to be an issue for AdaBoost, so I expect it to perform not much better than logistic regression (if at all better) once tuned.

In [ ]:
count = 1
model = AdaBoostClassifier()
acc_bal_dict = {}
wt_recall_dict = {}
feat_imp_dict = {}
for i in sets_to_test:
    acc_bal_scores = []
    wt_recall_scores = []
    for train, val in skf.split(i[0], i[1]):
        # Fit AdaBoost
        model.fit(i[0].iloc[train], i[1].iloc[train])
        # Get KNN score
        y_pred = model.predict(i[0].iloc[val])
        acc_bal_scores.append(balanced_accuracy_score(i[1].iloc[val], y_pred))
        wt_recall_scores.append(recall_score(i[1].iloc[val], y_pred, average = 'weighted'))
    acc_bal_dict[count] = acc_bal_scores
    wt_recall_dict[count] = wt_recall_scores
    feat_imp_dict[count] = model.feature_importances_
    count += 1

In [ ]:
acc_bal_dict

In [ ]:
wt_recall_dict

In [ ]:
scoring = ['recall', 'balanced_accuracy']
param_grid = {'n_estimators': range(40, 160, 20),
             'learning_rate': uniform(0.1, 1)}
rscv = RandomizedSearchCV(AdaBoostClassifier(), param_distributions = param_grid, n_iter = 50, scoring = scoring, 
                            cv = 5, refit = 'recall', error_score=0)

best_params_dict = {}
best_score_dict = {}
count = 1
for i in sets_to_test:
    search = rscv.fit(i[0], i[1])
    best_params_dict[count] = search.best_params_
    best_score_dict[count] = search.best_score_
    count += 1

In [ ]:
best_score_dict

Yup, AdaBoost wasn't as good as random forest. Definitely SLOW compared to other classifiers (though not terrible for this dataset - took a few minutes to do the randomized grid search for parameters).

# XGBoost and why I'm throwing all of this at the wall
Finally we get to the one that I was pretty sure I was going to end up with all along: XGBoost. But performance on the Pima Indianse Diabetes dataset isn't going to be the real test of this model, even if I can get a much better recall. What's going to be the real test is training and then pickling the models on a relatively minimal feature set (i.e. diastolic, BMI, age) and then testing them against the train subset of the Framingham data that I have. Whichever model is the most predictive with that data can be the one to finally get trained on the mixed training dataset and tested against the mixed test dataset for final scoring. If there isn't a clear winner, I can choose one that's fast.

In [33]:
# Getting baseline xgb performance
xgbc = xgb.XGBClassifier()
acc_bal_scores = []
wt_recall_scores = []
for train, val in skf.split(X_train_pd1, y_train_pd1):
    xgbc.fit(X_train_pd1.iloc[train], y_train_pd1.iloc[train])
    y_pred = xgbc.predict(X_train_pd1.iloc[val])
    acc_bal_scores.append(balanced_accuracy_score(y_train_pd1.iloc[val], y_pred))
    wt_recall_scores.append(recall_score(y_train_pd1.iloc[val], y_pred, average = 'weighted'))


In [34]:
acc_bal_scores

[0.5968070652173914,
 0.6355298913043479,
 0.5811820652173914,
 0.6222222222222222,
 0.6603174603174603]

In [ ]:
wt_recall_scores

OK, this is a pretty poor showing so far. First thing to try: correcting for imbalance in the dataset.

In [36]:
from sklearn.model_selection import GridSearchCV
param_grid = {'scale_pos_weights': [1, 2, 4, 8, 16]}
gscv = GridSearchCV(estimator = xgbc, param_grid = param_grid, cv = skf, scoring = 'recall')
gscv_result = gscv.fit(X_train_pd1, y_train_pd1)


[14:32:43] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1593723618214/work/src/learner.cc:480: 
Parameters: { scale_pos_weights } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:32:44] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1593723618214/work/src/learner.cc:480: 
Parameters: { scale_pos_weights } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:32:44] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1593723618214/work/src/learner.cc:480: 
Parameters: { scale_pos_weights } might not be used.

  This may not be accurate due to some parameters a

/Users/chesh1/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


[14:32:47] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1593723618214/work/src/learner.cc:480: 
Parameters: { scale_pos_weights } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [ ]:
gscv_result.best_score_

In [37]:
gscv_result.best_params_

{'scale_pos_weights': 1}

Well, if the best class weight is 1, that would seem to indicate that tuning the class weight hyperparameter is a non-starter. What else needs tuning?

In [ ]:
param_grid = {'learning_rate': [0.05, 0.1, 0.15, 0.2, 0.25, 0.3], 
             'eval_metric': ['error', 'auc']}
gscv = GridSearchCV(estimator = xgbc, param_grid = param_grid, cv = skf, scoring = 'recall')
gscv_result = gscv.fit(X_train_pd1, y_train_pd1)


In [ ]:
gscv_result.best_score_

In [ ]:
gscv_result.best_params_

In [38]:
param_grid = {'learning_rate': [0.01, 0.02, 0.03, 0.04, 0.05], 
             'eval_metric': ['error', 'auc']}
gscv = GridSearchCV(estimator = xgbc, param_grid = param_grid, cv = skf, scoring = 'recall')
gscv_result = gscv.fit(X_train_pd1, y_train_pd1)
gscv_result.best_score_

/Users/chesh1/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


0.5788979136805225

In [39]:
gscv_result.best_params_

{'eval_metric': 'error', 'learning_rate': 0.01}

In [ ]:
param_grid = {'learning_rate': [0.01],
              'n_estimators': [20, 40, 60, 80, 100],
              'max_depth': [3, 4, 5, 6, 7, 8, 9, 10]}
gscv = GridSearchCV(estimator = xgbc, param_grid = param_grid, cv = skf, scoring = 'recall')
gscv_result = gscv.fit(X_train_pd1, y_train_pd1)
gscv_result.best_score_

In [ ]:
gscv_result.best_params_